In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

education_df = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

In [13]:
education_df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


# 1. Determine all the variable types and find the fraction of the missing values for each variable.

Variable types

In [4]:
education_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

The fractions of missing values

In [10]:
education_df.isnull().sum()/education_df.count()*100

PRIMARY_KEY                       0.000000
STATE                             0.000000
YEAR                              0.000000
ENROLL                           21.399512
TOTAL_REVENUE                    16.562500
FEDERAL_REVENUE                  16.562500
STATE_REVENUE                    16.562500
LOCAL_REVENUE                    16.562500
TOTAL_EXPENDITURE                16.562500
INSTRUCTION_EXPENDITURE          16.562500
SUPPORT_SERVICES_EXPENDITURE     16.562500
OTHER_EXPENDITURE                21.399512
CAPITAL_OUTLAY_EXPENDITURE       16.562500
GRADES_PK_G                      13.115997
GRADES_KG_G                       9.705882
GRADES_4_G                        9.625276
GRADES_8_G                        9.625276
GRADES_12_G                       9.625276
GRADES_1_8_G                      9.625276
GRADES_9_12_G                     9.625276
GRADES_ALL_G                     13.115997
AVG_MATH_4_SCORE                178.358209
AVG_MATH_8_SCORE                180.451128
AVG_READING

# 2. Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?

Because there is no time dimension, missing data should just be filled with means on a state by state level. However, the average grade data has more null values than real values, so filling the data with mean values will be innapropriate and missing values should just be dropped. 

In [72]:
#Create df of States with average values for each variable
#Create copy of df to avoid changing original data
#Loop through education_df, if null replace with mean value for that variable for that state
#Do not loop through the average grade data

state_means = education_df.groupby('STATE').mean()
df2 = education_df.copy(deep=True)

for i in range(len(df2.columns)-4):
    for ii in range(len(df2.iloc[:,i])):
        if pd.isnull(df2.iloc[ii,i]):
            df2.iloc[ii,i] = state_means.loc[df2.loc[ii, 'STATE']][i]
    

In [53]:
df2.isnull().sum()/df2.count()*100

PRIMARY_KEY                       0.000000
STATE                             0.000000
YEAR                              0.000000
ENROLL                           12.096168
TOTAL_REVENUE                    12.096168
FEDERAL_REVENUE                  12.096168
STATE_REVENUE                    12.096168
LOCAL_REVENUE                    12.096168
TOTAL_EXPENDITURE                12.096168
INSTRUCTION_EXPENDITURE          12.096168
SUPPORT_SERVICES_EXPENDITURE     12.096168
OTHER_EXPENDITURE                 2.825637
CAPITAL_OUTLAY_EXPENDITURE        0.268817
GRADES_PK_G                       0.268817
GRADES_KG_G                       0.268817
GRADES_4_G                        0.268817
GRADES_8_G                        0.268817
GRADES_12_G                       0.268817
GRADES_1_8_G                      0.674764
GRADES_9_12_G                     1.982228
GRADES_ALL_G                      4.189944
AVG_MATH_4_SCORE                178.358209
AVG_MATH_8_SCORE                180.451128
AVG_READING

This processed removed almost all of the NaN values from the grades categories, however the enrollment, revenue, and expenditure values still have ~12% missing values. Any values remaining null must be from states that never had data collected for these variables and should probably be ignored in any analysis.

In [94]:
nan_states = []

for i in range(len(state_means.iloc[:])):
    for ii in range(len(state_means.iloc[i,:])):
        if pd.isnull(state_means.iloc[i,ii]):
            nan_states.append(state_means.index[i])

set(nan_states)

{'AMERICAN_SAMOA',
 'AS',
 'BI',
 'BIE',
 'BUREAU_OF_INDIAN_AFFAIRS',
 'BUREAU_OF_INDIAN_EDUCATIO',
 'BUREAU_OF_INDIAN_EDUCATION',
 'COMMONWEALTH_OF_MARIANAS',
 'DD',
 'DEPARTMENT_OF_DEFENSE',
 'DEPARTMENT_OF_DEFENSE_EDUCATION_ACTIVITY',
 'DOD',
 'DOD_(OVERSEAS_AND_DOMESTIC_COMBINED)',
 'DOD_-_DOMESTIC',
 'DOD_-_FOREIGN',
 'DOD_-_OVERSEAS',
 'DOD_DOMESTIC',
 'DOD_OVERSEAS',
 'GU',
 'GUAM',
 'MARIANAS',
 'MP',
 'NORTHERN_MARIANAS',
 'NORTHERN_MARIANA_ISLANDS',
 'PR',
 'PUERTO_RICO',
 'U.S._VIRGIN_ISLANDS',
 'VI',
 'VIRGIN_ISLANDS'}

By checking the states who still have null values, it is clear that they are non-typical small states and the fact that data is missing from them is not surprising.

# 3. Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using only the observations for that specific year.

Rather than filling in missing values by using the means of each state, I will impute them with the means for each year. 

In [95]:
#Create df of Years with average values for each variable
#Create copy of df to avoid changing original data
#Loop through education_df, if null replace with mean value for that variable for that state
#Do not loop through the average grade data

year_means = education_df.groupby('YEAR').mean()
df3 = education_df.copy(deep=True)

for i in range(len(df3.columns)-4):
    for ii in range(len(df3.iloc[:,i])):
        if pd.isnull(df3.iloc[ii,i]):
            df3.iloc[ii,i] = year_means.loc[df3.loc[ii, 'YEAR']][i]

In [96]:
df3.isnull().sum()/df3.count()*100

PRIMARY_KEY                       0.000000
STATE                             0.000000
YEAR                              0.000000
ENROLL                            3.539209
TOTAL_REVENUE                     3.539209
FEDERAL_REVENUE                   3.539209
STATE_REVENUE                     3.539209
LOCAL_REVENUE                     3.539209
TOTAL_EXPENDITURE                 3.899721
INSTRUCTION_EXPENDITURE           3.539209
SUPPORT_SERVICES_EXPENDITURE      3.539209
OTHER_EXPENDITURE                 3.539209
CAPITAL_OUTLAY_EXPENDITURE        3.539209
GRADES_PK_G                       7.338129
GRADES_KG_G                       7.338129
GRADES_4_G                        7.338129
GRADES_8_G                        7.338129
GRADES_12_G                       7.338129
GRADES_1_8_G                      4.555011
GRADES_9_12_G                     4.555011
GRADES_ALL_G                      6.343550
AVG_MATH_4_SCORE                178.358209
AVG_MATH_8_SCORE                180.451128
AVG_READING

This tecnique imputed much more data to the enroll, revenue, and expenditure variables. However, the grades had slightly higher null values remaining than before. Let's check which years were missing data.

In [97]:
nan_years = []

for i in range(len(year_means.iloc[:])):
    for ii in range(len(year_means.iloc[i,:])):
        if pd.isnull(year_means.iloc[i,ii]):
            nan_years.append(year_means.index[i])

set(nan_years)

{1992,
 1993,
 1994,
 1995,
 1997,
 1998,
 1999,
 2001,
 2002,
 2004,
 2006,
 2008,
 2010,
 2012,
 2014,
 2016,
 2017}

# 4. This time, fill in the missing values using interpolation (extrapolation).

I will use interpolation on the missing average grade variables as these data were collected only during select years. I will use a linear interpolation to estimate the average grades between years.

In [109]:
#Check the state of the raw average grades data per year 

avg_by_year = education_df.groupby(['YEAR'])['AVG_MATH_4_SCORE', 
                               'AVG_MATH_8_SCORE', 
                               'AVG_READING_4_SCORE',
                               'AVG_READING_8_SCORE'].mean()
avg_by_year

,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
YEAR,,,,
1992,218.410116,266.360319,212.712256,263.307067
1993,NaN,NaN,NaN,NaN
1994,NaN,NaN,NaN,NaN
1995,NaN,NaN,NaN,NaN
1996,221.479509,270.182528,214.587049,261.412097
1997,NaN,NaN,NaN,NaN
1998,NaN,NaN,NaN,NaN
1999,NaN,NaN,NaN,NaN
2000,224.803526,273.028605,217.612609,262.687292


In [112]:
avg_by_year = avg_by_year.interpolate(method='linear')
avg_by_year

,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
YEAR,,,,
1992,218.410116,266.360319,212.712256,263.307067
1993,219.177464,267.315871,213.180954,262.833324
1994,219.944813,268.271424,213.649652,262.359582
1995,220.712161,269.226976,214.118351,261.885840
1996,221.479509,270.182528,214.587049,261.412097
1997,222.310513,270.894047,215.343439,261.730896
1998,223.141517,271.605566,216.099829,262.049695
1999,223.972521,272.317085,216.856219,262.368493
2000,224.803526,273.028605,217.612609,262.687292


In [145]:
#Now the interpolated values need to be inserted into a copy of the original df to compare changes in nan values
df4 = education_df.copy(deep=True)

for i in range(len(df3)):
    for ii in range(len(df3.iloc[i,-4:])):
        if pd.isnull(df3.iloc[i,ii-4]):
            df4.iloc[i,ii-4] = avg_by_year.loc[df4.loc[i, 'YEAR']][ii]

In [146]:
df4.isnull().sum()/df4.count()*100

PRIMARY_KEY                      0.000000
STATE                            0.000000
YEAR                             0.000000
ENROLL                          21.399512
TOTAL_REVENUE                   16.562500
FEDERAL_REVENUE                 16.562500
STATE_REVENUE                   16.562500
LOCAL_REVENUE                   16.562500
TOTAL_EXPENDITURE               16.562500
INSTRUCTION_EXPENDITURE         16.562500
SUPPORT_SERVICES_EXPENDITURE    16.562500
OTHER_EXPENDITURE               21.399512
CAPITAL_OUTLAY_EXPENDITURE      16.562500
GRADES_PK_G                     13.115997
GRADES_KG_G                      9.705882
GRADES_4_G                       9.625276
GRADES_8_G                       9.625276
GRADES_12_G                      9.625276
GRADES_1_8_G                     9.625276
GRADES_9_12_G                    9.625276
GRADES_ALL_G                    13.115997
AVG_MATH_4_SCORE                 0.000000
AVG_MATH_8_SCORE                 0.000000
AVG_READING_4_SCORE              0

This method succesfully removed all of the NaN values for the average variables. However, it may not be appropriate to impute mean variables per year to every state.

# 5. Compare your results for the 2nd, 3rd, and 4th questions. Do you find any meaningful differences?

For a quick comparison, I will plot the mean values of enroll, total revenue, grades 1_8_g, and avg math 8 score for each solution sorted by the top 5 states then years

In [162]:
print('Output from question 2')
print(df2.groupby(['STATE'])['ENROLL', 
                        'TOTAL_REVENUE', 
                        'GRADES_1_8_G',
                        'AVG_MATH_8_SCORE'].mean().sort_values(by='ENROLL', ascending=False)[0:5])
print('\nOutput from question 3')
print(df3.groupby(['STATE'])['ENROLL', 
                        'TOTAL_REVENUE', 
                        'GRADES_1_8_G',
                        'AVG_MATH_8_SCORE'].mean().sort_values(by='ENROLL', ascending=False)[0:5])
print('\nOutput from question 4')
print(df4.groupby(['STATE'])['ENROLL', 
                        'TOTAL_REVENUE', 
                        'GRADES_1_8_G',
                        'AVG_MATH_8_SCORE'].mean().sort_values(by='ENROLL', ascending=False)[0:5])

Output from question 2
                  ENROLL  TOTAL_REVENUE  GRADES_1_8_G  AVG_MATH_8_SCORE
STATE                                                                  
CALIFORNIA  5.933285e+06   5.484420e+07  3.845589e+06        269.355244
TEXAS       4.277311e+06   3.601919e+07  2.785857e+06        280.455403
NEW_YORK    2.736288e+06   4.144276e+07  1.701955e+06        278.103480
FLORIDA     2.460021e+06   2.039945e+07  1.599173e+06        273.882398
ILLINOIS    2.002301e+06   2.073143e+07  1.271812e+06        280.709182

Output from question 3
                  ENROLL  TOTAL_REVENUE  GRADES_1_8_G  AVG_MATH_8_SCORE
STATE                                                                  
CALIFORNIA  5.945343e+06    55799199.28  3.702493e+06        269.355244
TEXAS       4.315420e+06    36879455.60  2.678940e+06        280.455403
NEW_YORK    2.748181e+06    42385624.88  1.636678e+06        278.103480
FLORIDA     2.486441e+06    20865131.36  1.540427e+06        273.882398
ILLINOIS    2.026

While this quick comparison certainly does not capture the majority of the data, it gives an idea about how the different data cleaning processes affected the top 5 states. The first solution, which averaged missing values on a state level shows the greatest changes for the first 3 variables, whereas solutions 2 and 3 are identical for these variables. Solution 3 has varying average grade scroes because the values are now being averaged over the interpolated values that were imputed.

In my opinion, the best method would be to impute new values averaged over the individual states. If states are stil missing data they should be removed for that particular variable. For the average scores which are only collected every few years, values should be imputed through interpolation for each year and for each state.